# Notebook to generate opers module

In [11]:
import project

from mintalib import core

from inspect import Signature, Parameter



In [12]:
def core_functions(first_param=('series', 'prices')):
    """ list of core functions """

    result = []
    
    for k, v in vars(core).items():
        if not k.isupper():
            continue

        if isinstance(v, type) or not callable(v):
            continue

        params = list(Signature.from_callable(v).parameters)

        if params[0] not in first_param:
            continue

        result.append(k)

    return result


print(core_functions())

['AVGPRICE', 'TYPPRICE', 'WCLPRICE', 'MIDPRICE', 'PRICE', 'CROSSOVER', 'CROSSUNDER', 'FLAG_ABOVE', 'FLAG_BELOW', 'INVERT_FLAG', 'UPDOWN_FLAG', 'LOG', 'EXP', 'ROC', 'DIFF', 'MIN', 'MAX', 'SUM', 'MAD', 'STDEV', 'SMA', 'EMA', 'RMA', 'WMA', 'DEMA', 'TEMA', 'MA', 'RSI', 'PLUSDI', 'MINUSDI', 'ADX', 'TRANGE', 'ATR', 'NATR', 'LATR', 'PSAR', 'CCI', 'CMF', 'MFI', 'BOP', 'BBANDS', 'KELTNER', 'EFFICIENCY_RATIO', 'KAMA', 'MACD', 'PPO', 'SLOPE', 'CURVE', 'STOCH', 'STREAK_UP', 'STREAK_DOWN', 'EVAL']


In [13]:
from mintalib.model import Operand
from mintalib.core import wrap_function


@wrap_function(core.EMA)
class EMA(Operand):
    same_scale = True

    def __init__(self, period: int, *, item: str=None):
        self.period = period
        self.item = item

    def __call__(self, series):
        return core.EMA(series, period=self.period)


In [14]:
import inspect


def make_operand(func, name: str = None, verbose=False):
    if name is None:
        name = func.__name__.upper()

    fname = func.__qualname__
    module = func.__module__.rpartition('.')[2]
    qname = f"{module}.{fname}"

    indent = " " * 4
    indent2 = " " * 8

    signature = Signature.from_callable(func)
    parameters = list(signature.parameters.values())

    if verbose:
        print("signature", signature)
        print("parameters", parameters)

    newparams = []
    ftype = parameters[0].name

    def annotate(param):
        if param.name in ['period']:
            return param.replace(annotation=int)

        if param.annotation == 'str':
            return param.replace(annotation=str)

        if type(p.default) in (int, float, bool):
            return param.replace(annotation=type(p.default))
        
        return param

    for i, p in enumerate(parameters):
        if i == 0:
            p = Parameter('self', Parameter.POSITIONAL_OR_KEYWORD)
        else:
            p = annotate(p)
        newparams.append(p)

    if verbose:
        print("newparams", newparams)

    newsig = Signature(newparams)

    def argument(arg):
        return ftype if arg == 'self' else f"{arg}=self.{arg}"

    args = [argument(p.name) for p in newparams]
    args = ", ".join(args)

    code = ""
    #code = "# noinspection PyPep8Naming\n"

    code += f"@wrap_function({qname})\n"
    code += f"class {name}(Operand):\n"
    
    if len(newparams) > 1:
        code += indent + f"def __init__" + str(newsig) + ":\n"
        for p in newparams[1:]:
            name = p.name
            code += indent2 + f"self.{name} = {name}\n"
        code += "\n"

    code += indent + f"def __call__(self, {ftype}):\n"
    code += indent2 + f"return {qname}({args})\n"

    return code


print(make_operand(core.EMA))

@wrap_function(core.EMA)
class EMA(Operand):
    def __init__(self, period: int, *, adjust: bool = False, item: str = None):
        self.period = period
        self.adjust = adjust
        self.item = item

    def __call__(self, series):
        return core.EMA(series, period=self.period, adjust=self.adjust, item=self.item)


In [15]:
def make_opers():
    buffer = "# Do not edit. File was generated by make-opers!\n\n"

    buffer += "from . import core\n"
    buffer += "from .model import Operand\n"
    buffer += "from .core import wrap_function\n"
    buffer += "\n"
    buffer += "nan: float = float('NAN')\n"
    buffer += "\n\n"

    names = core_functions()

    for name in names:
        func = getattr(core, name)
        text = make_operand(func)
        buffer += text + "\n\n"

    buffer += "__all__ = [n for n in dir() if n.isupper()]\n"

    return buffer

code = make_opers()
print(code[:256], "...")



# Do not edit. File was generated by make-opers!

from . import core
from .model import Operand
from .core import wrap_function

nan: float = float('NAN')


@wrap_function(core.AVGPRICE)
class AVGPRICE(Operand):
    def __call__(self, prices):
        retu ...


In [16]:
import importlib.util

def new_module(name):
    spec = importlib.util.spec_from_loader(name, None)
    return importlib.util.module_from_spec(spec)

code = make_opers()

indicators = new_module("mintalib.opers")

exec(code, indicators.__dict__)

dir(indicators)

['ADX',
 'ATR',
 'AVGPRICE',
 'BBANDS',
 'BOP',
 'CCI',
 'CMF',
 'CROSSOVER',
 'CROSSUNDER',
 'CURVE',
 'DEMA',
 'DIFF',
 'EFFICIENCY_RATIO',
 'EMA',
 'EVAL',
 'EXP',
 'FLAG_ABOVE',
 'FLAG_BELOW',
 'INVERT_FLAG',
 'KAMA',
 'KELTNER',
 'LATR',
 'LOG',
 'MA',
 'MACD',
 'MAD',
 'MAX',
 'MFI',
 'MIDPRICE',
 'MIN',
 'MINUSDI',
 'NATR',
 'Operand',
 'PLUSDI',
 'PPO',
 'PRICE',
 'PSAR',
 'RMA',
 'ROC',
 'RSI',
 'SLOPE',
 'SMA',
 'STDEV',
 'STOCH',
 'STREAK_DOWN',
 'STREAK_UP',
 'SUM',
 'TEMA',
 'TRANGE',
 'TYPPRICE',
 'UPDOWN_FLAG',
 'WCLPRICE',
 'WMA',
 '__all__',
 '__annotations__',
 '__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'core',
 'nan',
 'wrap_function']

In [17]:
outfile = project.pkgdir / "opers.py"

print(f"Updating {outfile.name} ...")

outfile.write_text(code)


Updating opers.py ...


13958